In [2]:
search_for = 132
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0022754669189453125
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 1268606955
type: [1, 1, 1, 1, 132] 132
cases of this type: 303595776
100000 38.41101139423956
200000 72.99236449814447
300000 85.92739086701297
400000 99.22840998050647
500000 101.47958474088144
600000 103.47412875627049
700000 99.13336425320294
800000 96.38446524857888
900000 96.00768409146914
1000000 98.49511845496791
1100000 96.28999936797723
1200000 26.998007521450784
1300000 96.84031048986884
1400000 95.89146867819152
1500000 94.90353741978284
1600000 92.40270255665685
1700000 95.06558340802971
1800000 99.29338195419633
1900000 96.37654454452341
2000000 95.0314357481718
2100000 94.90211602718264
2200000 96.81283649462775
2300000 75.26202049010399
2400000 46.359434952352245
2500000 95.4201980011259
2600000 92.73747072075543
2700000 93.8393971690468
2800000 92.21974833151477
2900000 97.49666291624486
3000000 94.209504136725

29600000 101.05436047487609
29700000 69.16295839329405
29800000 77.44547449670237
29900000 104.23074272040172
30000000 104.433115852572
30100000 102.06055542312562
30200000 43.46978403940968
30300000 105.15879110534217
30400000 104.44573151408777
30500000 106.72893187269862
30600000 116.98741256163385
30700000 111.55359846424129
30800000 99.32808322245897
30900000 100.32381609341039
31000000 93.58667925821256
31100000 27.039957023831604
31200000 100.06906913537465
31300000 97.00840105974753
31400000 98.61312230782768
31500000 101.2326401826328
31600000 101.93667589157312
31700000 100.05351974710237
31800000 105.61001073047994
31900000 102.45220348236145
32000000 58.5742092066682
32100000 89.68731924735893
32200000 109.44106383144559
32300000 100.21213544778105
32400000 102.11361911583556
32500000 34.38009342807277
32600000 101.16900887995337
32700000 103.15822265277805
32800000 97.80254592886632
32900000 101.80997019877559
33000000 98.93240438601116
33100000 95.7692989415297
33200000 9

59700000 98.94219877867629
59800000 109.86481630144672
59900000 96.05977368590692
60000000 101.68001396663183
60100000 99.0372909419727
60200000 105.18307839262842
60300000 37.31233575201998
60400000 100.1715736055982
60500000 91.98253253120372
60600000 95.50409964467764
60700000 89.73879642131124
60800000 86.85574005384667
60900000 95.12959444436042
61000000 23.97928253064985
61100000 94.69748394842345
61200000 91.65880628191596
61300000 86.14415648770263
61400000 92.30548371227691
61500000 90.01996880158667
61600000 99.44081747326616
61700000 36.77215257738395
61800000 99.4630592367619
61900000 107.52432901168814
62000000 96.17908639315614
62100000 104.71224969502589
62200000 100.52572505411491
62300000 105.05880602668277
62400000 104.68501535648701
62500000 101.43020420048528
62600000 50.73723579711765
62700000 90.51430189750215
62800000 94.93203209022708
62900000 98.37964896651769
63000000 86.94890806141227
63100000 95.42597480045711
63200000 96.57721710322673
63300000 23.427424950

90000000 91.82961100407267
90100000 80.81158378825717
90200000 78.84132334668247
90300000 78.08220091654759
90400000 47.01422269935385
90500000 58.69614561953847
90600000 80.96097202765122
90700000 75.37362351142642
90800000 80.05323417987366
90900000 19.974014860152888
91000000 78.16985864738106
91100000 74.17012028407983
91200000 80.66806480663848
91300000 71.21772364528518
91400000 45.732352646663635
91500000 76.14046292252597
91600000 86.74298620920695
91700000 78.76314309389905
91800000 88.20444996184717
91900000 77.38081421900127
92000000 81.21715347191756
92100000 82.42754930100598
92200000 82.89002162027751
92300000 78.71164054808303
92400000 78.92326122757243
92500000 70.48531040442079
92600000 77.32574579717921
92700000 56.297085596013375
92800000 46.077828474232824
92900000 71.47596034767442
93000000 72.58583168170351
93100000 73.35241556992783
93200000 17.813537017917692
93300000 87.36510476714652
93400000 76.9802075809538
93500000 71.37003122849472
93600000 43.503138397818

119600000 47.314831849792974
119700000 60.88770998780752
119800000 81.30296123723028
119900000 42.88743264185029
120000000 45.18218193802441
120100000 68.69784405872406
120200000 72.91656198930488
120300000 48.380563076747045
120400000 43.49181278308386
120500000 51.27278534370099
120600000 28.305152742662834
120700000 63.45336651800753
120800000 18.988904328154423
120900000 76.6630517861766
121000000 28.23097739668228
121100000 30.137212159416986
121200000 53.715747147538096
121300000 44.905196005058706
121400000 42.236601053842946
121500000 47.257032416476335
121600000 43.56222972438285
121700000 49.10450433129267
121800000 62.513404866289946
121900000 42.997257222621556
122000000 57.11218594285382
122100000 54.99315910385111
122200000 62.40281251629342
122300000 43.613140373858904
122400000 40.668537785923235
122500000 38.82180127194609
122600000 76.15205568530824
122700000 40.502872894328654
122800000 35.80306092014088
122900000 18.160533736682336
123000000 37.268097785598925
12310

...

1204100000 2.4202924505461056
1204200000 3.3313078859227203
1204300000 2.6922908953843745
1204400000 2.3025527057621016
1204500000 3.1495697747558355
1204600000 2.788891713373697
1204700000 2.336097714389877
1204800000 3.2245118137376907
1204900000 2.3436131085667378
1205000000 2.719622430397835
1205100000 3.4353482517080405
1205200000 2.6242353006998864
1205300000 2.745202441648096
1205400000 3.3844861546007095
1205500000 2.1598090548115505
1205600000 2.8590702382545774
1205700000 3.313462217832397
1205800000 1.754552901409977
1205900000 3.1642316236601276
1206000000 3.3557037645998595
1206100000 1.881042184785972
1206200000 3.0748355202044184
1206300000 3.4066861757701634
1206400000 1.7512482346963851
1206500000 3.090777299531612
1206600000 2.7355886579478677
1206700000 2.0080176808526446
1206800000 3.3169286640280617
1206900000 3.0735646780308885
1207000000 2.723092885963354
1207100000 1.8748900408557685
1207200000 0.5610205257219076
1207300000 0.5385436030966374
1207400000 0.633945

1231100000 0.460362416783909
1231200000 0.777387071249972
1231300000 0.7069634645155304
1231400000 0.4194605906471577
1231500000 0.483839214381665
1231600000 0.6131735981578993
1231700000 0.29971308571219446
1231800000 0.4911611162700554
1231900000 0.46384772618216935
1232000000 0.42384854717830817
1232100000 0.5508681173771421
1232200000 0.3748408383759359
1232300000 0.4741907690138684
1232400000 0.4809192769551277
1232500000 0.3673686216055486
1232600000 0.3944533038557404
1232700000 0.6553948076958557
1232800000 0.38151619803249837
1232900000 0.4310760306489666
best so far: 0
type: [2, 1, 1, 6, 11] 132
cases of this type: 3162456
1233000000 0.5405237523596287
1233100000 0.35759767341138254
1233200000 0.5751864142003226
1233300000 0.6227699051561058
1233400000 0.37407688606888384
1233500000 0.638342446613762
1233600000 0.8885066622808476
1233700000 0.47786350291728313
1233800000 0.5574159276699125
1233900000 0.593290486863196
1234000000 0.3408680330054462
1234100000 0.702260318323363

1252500000 0.22191658696499467
1252600000 0.25236557969019147
1252700000 0.1859297284480797
1252800000 0.19444194951817392
1252900000 0.24203731237666806
1253000000 0.2051500395588676
1253100000 0.18448179178133606
1253200000 0.23710510023932985
best so far: 0
type: [2, 2, 1, 3, 11] 132
cases of this type: 790614
1253300000 0.22777289700564743
1253400000 0.18031720929645498
1253500000 0.12224896289530397
1253600000 0.13518329113522837
1253700000 0.11094395912214121
1253800000 0.11179495914740033
1253900000 0.1314559737842381
1254000000 0.1682570735257268
best so far: 0
type: [2, 2, 1, 11, 3] 132
cases of this type: 215622
1254100000 0.21080223181962635
1254200000 0.19396090853110617
best so far: 0
type: [2, 2, 1, 33, 1] 132
cases of this type: 71874
1254300000 0.07670422905895113
best so far: 0
type: [2, 2, 3, 1, 11] 132
cases of this type: 263538
1254400000 0.10896903652407064
1254500000 0.05844980897569988
best so far: 0
type: [2, 2, 3, 11, 1] 132
cases of this type: 23958
best so fa

best so far: 0
type: [3, 2, 2, 11, 1] 132
cases of this type: 10648
best so far: 0
type: [3, 2, 11, 1, 2] 132
cases of this type: 3872
best so far: 0
type: [3, 2, 11, 2, 1] 132
cases of this type: 1936
best so far: 0
type: [3, 2, 22, 1, 1] 132
cases of this type: 968
best so far: 0
type: [3, 4, 1, 1, 11] 132
cases of this type: 58564
best so far: 0
type: [3, 4, 1, 11, 1] 132
cases of this type: 5324
best so far: 0
type: [3, 4, 11, 1, 1] 132
cases of this type: 484
best so far: 0
type: [3, 11, 1, 1, 4] 132
cases of this type: 2816
best so far: 0
type: [3, 11, 1, 2, 2] 132
cases of this type: 1408
best so far: 0
type: [3, 11, 1, 4, 1] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 1, 2] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 2, 1] 132
cases of this type: 352
best so far: 0
type: [3, 11, 4, 1, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 1, 1, 2] 132
cases of this type: 352
best so far: 0
type: [3, 22, 1, 2, 1] 132
cases of this type: 176
b

cases of this type: 16
best so far: 0
type: [33, 2, 2, 1, 1] 132
cases of this type: 8
best so far: 0
type: [33, 4, 1, 1, 1] 132
cases of this type: 4
best so far: 0
type: [44, 1, 1, 1, 3] 132
cases of this type: 81
best so far: 0
type: [44, 1, 1, 3, 1] 132
cases of this type: 27
best so far: 0
type: [44, 1, 3, 1, 1] 132
cases of this type: 9
best so far: 0
type: [44, 3, 1, 1, 1] 132
cases of this type: 3
best so far: 0
type: [66, 1, 1, 1, 2] 132
cases of this type: 16
best so far: 0
type: [66, 1, 1, 2, 1] 132
cases of this type: 8
best so far: 0
type: [66, 1, 2, 1, 1] 132
cases of this type: 4
best so far: 0
type: [66, 2, 1, 1, 1] 132
cases of this type: 2
best so far: 0
type: [132, 1, 1, 1, 1] 132
cases of this type: 1
best so far: 0
all cases: 1268606955
best: 0
